# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: This is a classification problem, as the goal suggests splitting students into two groups, one where the students have a high likelihood that they will fail to graduate if they do not get early intervention, and the other group who are the students who do not belong to the first group, so it is clear this is a binary classification problem and not a regression problem.**

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [9]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print("Student data read successfully!")

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [10]:
# TODO: Calculate number of students
# TODO: Calculate number of features
n_students, n_features = np.shape(student_data)

# TODO: Calculate passing students
n_passed = np.shape(student_data[student_data["passed"] == "yes"])[0]

# TODO: Calculate failing students
n_failed = n_students-n_passed

# TODO: Calculate graduation rate
grad_rate = float(n_passed)/float(n_students)*100.00

# Print the results
print("Total number of students: {}".format(n_students))
print("Number of features: {}".format(n_features))
print("Number of students who passed: {}".format(n_passed))
print("Number of students who failed: {}".format(n_failed))
print("Graduation rate of the class: {:.2f}%".format(grad_rate))

Total number of students: 395
Number of features: 31
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [11]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print("Feature columns:\n{}".format(feature_cols))
print("\nTarget column: {}".format(target_col))

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print("\nFeature values:")
print(X_all.head())

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [12]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print("Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns)))

#OnehotEncodes y variable
#y_all = y_all.replace(['yes', 'no'], [1, 0])


Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [13]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))
print(X_train.head())

Training set has 300 samples.
Testing set has 95 samples.
     school_GP  school_MS  sex_F  sex_M  age  address_R  address_U  \
50         1.0        0.0    1.0    0.0   16        0.0        1.0   
140        1.0        0.0    0.0    1.0   15        0.0        1.0   
188        1.0        0.0    1.0    0.0   17        0.0        1.0   
176        1.0        0.0    1.0    0.0   16        0.0        1.0   
172        1.0        0.0    0.0    1.0   17        0.0        1.0   

     famsize_GT3  famsize_LE3  Pstatus_A    ...     higher  internet  \
50           0.0          1.0        0.0    ...          1         1   
140          1.0          0.0        0.0    ...          1         1   
188          1.0          0.0        1.0    ...          1         1   
176          1.0          0.0        0.0    ...          1         1   
172          0.0          1.0        0.0    ...          1         1   

     romantic  famrel  freetime  goout  Dalc  Walc  health  absences  
50          0    

## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: 

1) Logistic Regression : Logistic regression is a common analysis tool used for Genome-Wide Association Studies (GWAS) when the response variable of interest is qualitative for example disease/normal, Blood group A/B/AB/O etc. It comes as one of the standard tools in most GWAS packages (e.g. PLINK).
    - Strength of logistic regression is that Logistic regression models are easy to interpret and can be implemented on large number of features and is fast and reliable when the dimension gets large. Additionally  there are lots of techniques developed to do subset selection or stepwise feature selection. It also lends itself to regularization which can help to decrease overfitting. It provides a probabilistic prediction as to the maximum likelihood of prediction of one category vs another.
    - Weakness of the model is that large sample sizes are required for logistic regression to provide similar numbers in each categories of the response variable, and performance is poor when number of samples are not equally represented between categories. The more features, the larger the sample size required to obtain generalizable results. Ultimately logistic regression is still a linear classifier and though more complex exponents can be added to the features, it is limited in modelling highly complex non linear relationships between features and the response variable. Can only work with numerical data, and so categorical data needs to be converted to numeric with one-hot encoding.
    - logistic regression analysis is based on calculating the odds of not graduating i.e. the ratio of the probability of not graduating divided by the probability of graduating, which is perfect for this problem. Also since there are a comparable number of students likely to graduate and number of students not likely to graduate, a reasonable logistic model can be built. As the number of features are large logistic regression is helpful. 

2) Decision Trees: Decision trees work by segmenting the feature space into hyper-rectangles in high dimensions. Real world use of tree based classification for drug analysis in Pharmacology  https://www.karger.com/Article/Pdf/119068
    - Decision tree models are particularly useful with prediction with small number of features, and leads to a clearly visializable decision flow, which is very intuitive for human understanding. Can work with both numerical or categorical data, and doesnt necessarily need one hot encoding, and very little data preparation. It is very fast when the number of features are small.
    - Weakness of decision tree models, is that very complex trees can be constructed making them very prone to overfitting since they have high degrees of freedom. Decision boundaries are parallel to the axes, for each pair of features, and so a large number of boundaries are needed to fit complex relationships. 
    - It is useful in modeling this data, as it creates an easy to interpret tree, and since this data is a combination of categorical and numerical features, the model will be easy to build into a tree. 

3) Support Vector Machines : SVMs are yet another supervised learning methods which can be used for classification, the essence of which is to maximize the margins around the discriminant hyperplane. A real world application of SVM is seen in Financial time series prediction http://www.svms.org/finance/VanGestel-etal2001.pdf
    - Strengths of SVMs are that : SVMs are quiet versatile as domain knowledge can be introduced into the model by carefully designing Kernels. Regularization parameters allow for preventing overfitting of SVM models. Effective in complex high dimensional spaces. Uses a subset of training points in the decision function (support vectors), so it is also memory efficient, and since SVM is using the points along the seperation boundary, it is less sensitive to outliers.
    - If the number of features are larger than the number of observations, then it is also prone to overfitting. Choice of the kernel is a key determinant as to the effectiveness of the model, depending on the choice of kernel, it could result in both high bias as well as high variance models. SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation.
    - SVMs may be a good choice in this classification problem, as there are a sizable number of observations to be able to build generalizable SVM models, and the number of features seems to be smaller than the number of observations thus making it less prone to overfitting.
**

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [14]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print("Made predictions in {:.4f} seconds.".format(end - start))
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print("F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [15]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
from sklearn import linear_model
# from sklearn import model_B
from sklearn.tree import DecisionTreeClassifier
# from skearln import model_C
from sklearn import svm

# TODO: Initialize the three models
clf_A = linear_model.LogisticRegression(random_state=0)
clf_B = DecisionTreeClassifier(random_state=0)
clf_C = svm.SVC(random_state=0)

# TODO: Set up the training set sizes

X_train_100 = X_train[0:100]
y_train_100 = y_train[0:100]

X_train_200 = X_train[0:200]
y_train_200 = y_train[0:200]

X_train_300 = X_train[0:300]
y_train_300 = y_train[0:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)

train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)

train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0016 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 0.8800.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7914.
Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0008 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7786.
Training a SVC using a training set size of 100. . .
Trained model in 0.0011 seconds
Made predictions in 0.0007 seconds.
F1 score for training set: 0.8452.
Made predictions in 0.0007 seconds.
F1 score for test set: 0.7975.
Training a LogisticRegression using a training set size of 200. . .
Trained model in 0.0020 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8449.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.8252.
Training a DecisionTreeClassifier using a training set size

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Logistic Regression**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0016          |         0.0002         |       0.8800     |       0.7914    |
| 200               |         0.0020          |         0.0002         |       0.8449     |       0.8252    |
| 300               |         0.0032          |         0.0007         |       0.8341     |       0.7704    |

** Classifer 2 - Decision Trees**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0008          |         0.0002         |       1.0000     |        0.7786   |
| 200               |         0.0017          |         0.0005         |       1.0000     |        0.7132   |
| 300               |         0.0021          |         0.0004         |       1.0000     |        0.6780   |

** Classifer 3 - Support Vector Machines**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0011          |         0.0007         |       0.8452     |        0.7975   |
| 200               |         0.0038          |         0.0025         |       0.8528     |        0.8129   |
| 300               |         0.0070          |         0.0016         |       0.8596     |        0.7947   |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: In the experiments conducted above, logistic regression and SVM performed similarly, as the two methods had comparable performances between the training and the test sets. Given their similar performances, I chose Logistic regression to model this paricular problem of predicting student performance due to the following reasons:

    - Logistic regression is easy to administer, and it yields convenient probability scores for each obeservation, and many standard and custom performance metrics can be applied on this probability score to get a cutoff and in turn classify output in a way which best fits this particular problem. This also provides a convenient probabilistic framework for the model.
    - Multi-collinearity is not really an issue and can be countered with L2 regularization to an extent. While it is crucial to perform feature selection to remove highly collinear features while using SVMs.
    - SVM would perform very well especially with high dimensional data, but in this case as the number of features is not too high, logistic regression is easy to administer, and so will perform efficiently even with limited resources.
    - Also with SVM it is hard to find the right kernel, which is not an issue with logistic regression.
**

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: 
The core premise with logistic regression is that the input features are able to provide a reliable probability of how likely a given student is likely to fail, and thus will require intervention. 

The model is built by comparing the input features from historical examples of students who have passed and those who have failed. As each feature is able to provide some information as to the likelihood of the student passsing vs failing, and then combining the information from all the features to arrive at a best combination of features such that we can maximize our ability to predict the probability that a student will fail.

Once we have a predicted probability for each historical student example, we can then select an appropriate cutoff such that we can maximize the number of students who need intervention, thus able to have the best impact on the pass percentage of the cohort as a whole. However what this also allows us to do is to set up a cut off such that we can also rank the students such that if limited resources are available, intervention can be provided to the students that are most in need. 

Once we have a suitable cut off for probability we can now use this model on a new group of students, to select among them who needs intervention, with the limited resources available for the school, so as to maximize the pass percentage for this cohort.
**

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [41]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
#parameters = {'degree':[0,1,2,3],'C':[0.0000001, 0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10, 100],'kernel':['rbf', 'poly', 'linear', 'sigmoid']}
parameters = {'C':[0.01, 0.1, 1.0, 10.0, 100.0]}
# TODO: Initialize the classifier
#clf = svm.SVC(random_state=0)
clf = linear_model.LogisticRegression(random_state=0)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, greater_is_better=True, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, param_grid=parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)   

# Get the estimator
clf = grid_obj.best_estimator_
print(clf)
# Report the final F1 score for training and testing after parameter tuning
print("Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train)))
print("Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test)))

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Made predictions in 0.0004 seconds.
Tuned model has a training F1 score of 0.8139.
Made predictions in 0.0002 seconds.
Tuned model has a testing F1 score of 0.8079.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: The F1 score for the training set using the new model is 0.8139, and the test set is 0.8079. The F1 score for the training set using the untuned model is 0.8341 and test set is 0.7704. This suggests that the tuned model which has a C=0.01, has a regularization, which prevents overfitting of the model as suggested by the fact that the F1score of the new model on the training set went down. And also the model is more generalizable now as the F1score on the test set improved.**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.